<a href="https://colab.research.google.com/github/fralfaro/MAT306/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT306 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [ ]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_codigo.csv').rename(columns=str.lower)

df_codigos.head()

,codigo_iso,pais
0,AFG,Afghanistán
1,AGO,Angola
2,ALB,Albania
3,AND,Andorra
4,ARE,Emiratos Árabes Unidos


---
---



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



---

In [ ]:
# (a) Unir años en df_anio y normalizar a minúscula
df_anio = pd.concat([pd.read_csv(u) for u in archivos_anio], ignore_index=True).rename(columns=str.lower)
df_anio

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.50,59.0
1,AGO,2001,30.20,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN
...,...,...,...,...
3055,WSM,2019,18.25,22.0
3056,YEM,2019,61.66,168.0
3057,ZAF,2019,22.19,31.0
3058,ZMB,2019,36.38,119.0


---

In [20]:
# (b) Limpiar df_codigos (resolver códigos con 2 nombres de país)

# ver cuántos nombres distintos hay por codigo_iso
resumen = (
    df_codigos.groupby("codigo_iso")["pais"]
    .nunique()
    .reset_index(name="n_paises")
)


# detectar los que tienen más de 1 nombre
conflictivos = resumen.query("n_paises > 1")["codigo_iso"]
print(f"Codigos conflictivos: {conflictivos.tolist()} \n{'*'*10}NewLine{'*'*10}")


# Analicemos que nombres tienen esos códigos
print(
    df_codigos[df_codigos["codigo_iso"].isin(conflictivos)]
    .sort_values(["codigo_iso", "pais"])
)

Codigos conflictivos: ['ZWE'] 
**********NewLine**********
    codigo_iso      pais
179        ZWE  Zimbabue
180        ZWE      malo


In [23]:
# Eliminaremos el pais malo del df
wrong_index = df_codigos[(df_codigos['codigo_iso']=='ZWE') &
                         (df_codigos['pais']=='malo')].index

# si no está vacío, dropear
if not wrong_index.empty:
    df_codigos = df_codigos.drop(wrong_index)

# verificar que solo quedó el correcto
print(df_codigos[df_codigos['codigo_iso']=='ZWE'])

    codigo_iso      pais
179        ZWE  Zimbabue


---

In [25]:
# (c) Unir para obtener df final (solo coincidencias en ambas fuentes)
# Usaremos inner para que se quedan solo las filas que coincidan con "codigo_iso" en ambos DataFrames
df = pd.merge(df_anio, df_codigos, on="codigo_iso", how="inner")
df


,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.50,59.0,Afghanistán
1,AGO,2001,30.20,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos
...,...,...,...,...,...
3055,WSM,2019,18.25,22.0,Samoa
3056,YEM,2019,61.66,168.0,Yemen
3057,ZAF,2019,22.19,31.0,Sudáfrica
3058,ZMB,2019,36.38,119.0,Zambia


---
---


### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

---
**Estructura del DataFrame**

In [30]:
# ======= ESTRUCTURA =======
print(f"{'='*60}\nEstructura del DataFrame\n{'='*60}\n")
print("Filas (observaciones):", df.shape[0])
print("Columnas:", df.shape[1])
print("Nombres de columnas:", df.columns.tolist())
print("\nTipos de datos:\n", df.dtypes)

# ======= RESUMEN ESTADÍSTICO =======
print(f"{'='*60}\nResumen Estadístico\n{'='*60}\n")
print("\nResumen describe():\n", df.describe(include="all"))

print("\nValores de 'indice':")
print("  Mínimo:", df['indice'].min())
print("  Máximo:", df['indice'].max())
print("  Promedio:", df['indice'].mean())

print("\nPaís con menor indice:", df.loc[df['indice'].idxmin(), 'pais'])
print("País con mayor indice:", df.loc[df['indice'].idxmax(), 'pais'])

print("\nPaís con mejor ranking (1 = más alto):", df.loc[df['ranking'].idxmin(), 'pais'])
print("País con peor ranking:", df.loc[df['ranking'].idxmax(), 'pais'])

# ======= DATOS FALTANTES =======
print(f"{'='*60}\nDatos Faltantes\n{'='*60}\n")
print("\nValores nulos por columna:\n", df.isnull().sum())
print("\nProporción de nulos (%):\n", (df.isnull().mean() * 100).round(2))
print("\nColumnas con >30% nulos:\n", df.isnull().mean()[df.isnull().mean() > 0.3])

# ======= UNICIDAD Y DUPLICADOS =======
print(f"{'='*60}\nUnicidad y Duplicados\n{'='*60}\n")
print("\nPaíses distintos:", df['pais'].nunique())
print("Años distintos:", df['anio'].nunique())
print("Filas duplicadas:", df.duplicated().sum())

# ======= VALIDACIÓN CRUZADA =======
print(f"{'='*60}\nValidación Cruzada de Columnas\n{'='*60}\n")
# verificar si algún codigo_iso está asociado a más de un país
conflictos = df.groupby('codigo_iso')['pais'].nunique()
conflictivos = conflictos[conflictos > 1]
print("\nCódigos ISO asociados a más de un país:\n", conflictivos)


Estructura del DataFrame

Filas (observaciones): 3060
Columnas: 5
Nombres de columnas: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos de datos:
 codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object
Resumen Estadístico


Resumen describe():
        codigo_iso         anio        indice        ranking     pais
count        3060  3060.000000   2664.000000    2837.000000     3060
unique        180          NaN           NaN            NaN      179
top           AFG          NaN           NaN            NaN  Nigeria
freq           17          NaN           NaN            NaN       34
mean          NaN  2009.941176    205.782316     477.930913      NaN
std           NaN     5.786024   2695.525264    6474.935347      NaN
min           NaN  2001.000000      0.000000       1.000000      NaN
25%           NaN  2005.000000     15.295000      34.000000      NaN
50%           NaN  2009.000000     28.000000      70.00

---
---

### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [32]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america =  pd.DataFrame() # FIX ME

---
Solución **a)**

In [ ]:
# Lista de años ordenados
lista_años = sorted(df_america["anio"].unique())

for año in lista_años:
    df_temp = df_america[df_america["anio"] == año]

    # Filtrar solo filas con 'indice' válido (int o float y no NaN)
    df_IndexNotNull = df_temp[df_temp["indice"].apply(lambda x: isinstance(x, (int,float)))].dropna(subset=['indice'])

    if not df_IndexNotNull.empty:
        idx_min = df_IndexNotNull["indice"].idxmin()
        idx_max = df_IndexNotNull["indice"].idxmax()

        print(f"\nEn el año {año}")
        print(f"  El mejor país fue : {df_IndexNotNull.loc[idx_min, 'pais']} ({df_IndexNotNull.loc[idx_min, 'indice']})")
        print(f"  El peor país fue  : {df_IndexNotNull.loc[idx_max, 'pais']} ({df_IndexNotNull.loc[idx_max, 'indice']})")
    else:
        # Aquí informamos explícitamente que no hay valores válidos
        print(f"\nEn el año {año} no hay valores válidos de 'indice'")



En el año 2001
  El mejor país fue : Canadá (0.8)
  El peor país fue  : Cuba (90.3)

En el año 2002
  El mejor país fue : Trinidad y Tobago (1.0)
  El peor país fue  : Cuba (97.83)

En el año 2003
  El mejor país fue : Trinidad y Tobago (2.0)
  El peor país fue  : Argentina (35826.0)

En el año 2004
  El mejor país fue : Trinidad y Tobago (2.0)
  El peor país fue  : Cuba (87.0)

En el año 2005
  El mejor país fue : Bolivia (4.5)
  El peor país fue  : Cuba (95.0)

En el año 2006
  El mejor país fue : Canadá (4.88)
  El peor país fue  : Cuba (96.17)

En el año 2007
  El mejor país fue : Canadá (3.33)
  El peor país fue  : Cuba (88.33)

En el año 2008
  El mejor país fue : Canadá (3.7)
  El peor país fue  : Cuba (94.0)

En el año 2009
  El mejor país fue : Estados Unidos (6.75)
  El peor país fue  : Cuba (78.0)

En el año 2012
  El mejor país fue : Jamaica (9.88)
  El peor país fue  : Cuba (71.64)

En el año 2013
  El mejor país fue : Jamaica (10.9)
  El peor país fue  : Cuba (70.92)

En

---
Solución **b)**

In [39]:
# Dado que groupby necesita valores numéricos válidos, haremos el filtro de estos valores.
df_america_filtrado = df_america.dropna(subset=["indice"])  #Elimina filas donde el valor de indice es NaN, asumiremos que en esta columna indice siempre se trabaja con NaN o numméricos válidos.

# Países con mejor índice por año
mejores = df_america_filtrado.loc[df_america_filtrado.groupby("anio")["indice"].idxmin()]   #Guardamos el indice en el df de los paises con mejor indice por año
mejores = mejores.drop(["codigo_iso", "ranking"], axis=1) #Eliminamos las columnas innecesarias
mejores = mejores.rename(columns={"anio": "Año", "pais": "Mejor País", "indice": "Mejor Índice"}) #Renombramiento de Columnas

# 3. Países con peor índice por año
peores = df_america_filtrado.loc[df_america_filtrado.groupby("anio")["indice"].idxmax()]
peores = peores.drop(["codigo_iso", "ranking"], axis=1)
peores = peores.rename(columns={"anio": "Año", "pais": "Peor País", "indice": "Peor Índice"})

# 4. Combinar ambos resultados
extremos = pd.merge(mejores, peores, on="Año", how="outer").sort_values("Año")

print(extremos)


     Año  Mejor Índice         Mejor País  Peor Índice  Peor País
0   2001          0.80             Canadá        90.30       Cuba
1   2002          1.00  Trinidad y Tobago        97.83       Cuba
2   2003          2.00  Trinidad y Tobago     35826.00  Argentina
3   2004          2.00  Trinidad y Tobago        87.00       Cuba
4   2005          4.50            Bolivia        95.00       Cuba
5   2006          4.88             Canadá        96.17       Cuba
6   2007          3.33             Canadá        88.33       Cuba
7   2008          3.70             Canadá        94.00       Cuba
8   2009          6.75     Estados Unidos        78.00       Cuba
9   2012          9.88            Jamaica        71.64       Cuba
10  2013         10.90            Jamaica        70.92       Cuba
11  2014         10.99             Canadá        70.21       Cuba
12  2015         11.10         Costa Rica        70.23       Cuba
13  2017         11.93         Costa Rica        71.75       Cuba
14  2018  

---
---

### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?

**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





---
Solución **a)**

In [45]:
# Tabla pivote: países en filas, años en columnas, valores = indice máximo

pivot = pd.pivot_table(
    df, 
    index="pais",      # filas = países
    columns="anio",    # columnas = años
    values="indice",   # valores en la tabla = columna 'indice'
    aggfunc="max",     # si hay varios valores en un país/año, toma el máximo
    fill_value=0       # los espacios vacíos (NaN) se reemplazan con 0
)

pivot.head()




anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63


In [59]:
# Mayor y menor índice
max_val = pivot.values.max()
min_val = pivot.values[pivot.values > 0].min() # ignorar ceros

# mínimo de cada país
paises_min = pivot.loc[(pivot > 0).any(axis=1)].min(axis=1)

# países con el valor mínimo global
paises_minimos = pivot.index[(pivot == min_val).any(axis=1)].tolist()

# máximo de cada país
paises_max = pivot.max(axis=1)

# países con el valor máximo global
paises_maximos = pivot.index[(pivot == max_val).any(axis=1)].tolist()


print("País(es) con el mayor índice :", ", ".join(paises_maximos), " con ", max_val)
print("País(es) con el menor índice no Zero:", ", ".join(paises_minimos), " con ", min_val)

País(es) con el mayor índice : Kosovo  con  64536.0
País(es) con el menor índice no Zero: Austria, Dinamarca, Eslovaquia, Estonia, Finlandia, Irlanda, Islandia, Noruega, Países Bajos, Suiza  con  0.5


---
Solución **b)**

In [ ]:
# promedio de indice por año, ignorando los ceros que son "faltantes"
prom_anual = pivot.replace(0, np.nan).mean(axis=0)

anio_max = prom_anual.idxmax()
anio_min = prom_anual.idxmin()

print("Año con promedio más alto:", anio_max, "→", prom_anual.max())
print("Año con promedio más bajo:", anio_min, "→", prom_anual.min())

Año con promedio más alto: 2012 → 468.88350877192977
Año con promedio más bajo: 2002 → 25.879683544303795


---
Solución **c)**

In [ ]:
pivot_WithoutZeros = pivot.replace(0, np.nan)

# País con mayor variabilidad 
variabilidad = pivot_WithoutZeros.max(axis=1) - pivot_WithoutZeros.min(axis=1)

pais_var = variabilidad.idxmax()
val_var = variabilidad.max()

print("País con mayor variabilidad:", pais_var, "→", val_var)

País con mayor variabilidad: Kosovo → 46098.0


---
Solución **d)**

In [71]:
constantes = variabilidad[variabilidad == 0].index.tolist()
print("Países con índice constante:", constantes if constantes else "Ninguno")

Países con índice constante: ['Antigua y Barbuda', 'Granada']


---
Solución **e)**

In [70]:
# Son los que quedaron con toda la fila NaN después de limpiar
sin_datos = pivot_WithoutZeros[pivot_WithoutZeros.isna().all(axis=1)].index.tolist()
print("Países sin datos:", sin_datos if sin_datos else "Ninguno")

Países sin datos: Ninguno
